In [ ]:
!pip install pandas
!pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from IPython.display import display
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv('data/train.csv')
df.head(3)

In [ ]:
df.dtypes

In [ ]:
# remove nan values
# df_bs = df.dropna()

In [ ]:
# convert each object column to a category
category_mapping = {}
categorised_df = df.copy(deep=False)
for col in categorised_df.select_dtypes(include='object').columns:
    categorised_df[col] = categorised_df[col].astype('category')
    category_mapping[col] = {value: i for i, value in enumerate(categorised_df[col].unique())}
    categorised_df[col] = categorised_df[col].map(category_mapping[col])
print(category_mapping)
categorised_df.head(2)


In [ ]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(categorised_df.drop(columns=['outcome'])), columns=categorised_df.drop(columns=['outcome']).columns)
y = categorised_df['outcome']
display(X.head(2))
display(y.head(2))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
clf = RandomForestClassifier(n_estimators=200, max_depth=50, random_state=0)
clf.fit(X_train, y_train)
print(f"Train accuracy: {clf.score(X_train, y_train)}")
print(f"Test accuracy: {clf.score(X_test, y_test)}")

In [ ]:
model = RandomForestClassifier()
hp = {'max_depth': [10, 50, 100, 200, 500, 1000, 2000], 'n_estimators': [10, 50, 100, 200, 400, 600, 1000]}
hp = {
    'n_estimators': [10, 50, 100, 200], 
    'max_depth': [None, 5, 10, 20], 
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4], 
    'bootstrap': [True, False],
    # 'max_features': ['auto', 'sqrt']
}
random_rf = RandomizedSearchCV(estimator=model, param_distributions=hp,
                              n_iter=100, scoring='accuracy', cv=5,
                              refit=True, n_jobs=-1, verbose=1)
random_rf.fit(X_train, y_train)
print(f"Final Test accuracy: {random_rf.score(X_test, y_test)}")

In [ ]:
# create predictions for the test set
test_df = pd.read_csv('data/test.csv')
for col in test_df.select_dtypes(include='object').columns:
    test_df[col] = test_df[col].map(category_mapping[col])
scaled_test = pd.DataFrame(scaler.transform(test_df), columns=test_df.columns)
print("nan values in test set:", scaled_test.isna().sum().sum())
# print the nan values
scaled_test[scaled_test.isna().any(axis=1)]
# replace nan value with the mean of the column
scaled_test['pain'].fillna(scaled_test['pain'].mean(), inplace=True)
display(scaled_test.head(2))

predictions = random_rf.predict(scaled_test)
ids = test_df['id']
reverse_mapping = {v: k for k, v in category_mapping['outcome'].items()}
submission_df = pd.DataFrame({'id': ids, 'outcome': [reverse_mapping[prediction] for prediction in predictions]})
submission_df.head(10)


In [ ]:
submission_df.to_csv('data/submission_doug.csv', index=False)

## Trying with XGBoost

In [ ]:
!pip install xgboost

In [74]:
from xgboost import XGBRegressor, XGBClassifier
# Define the model
# model = XGBRegressor(n_estimators=1000, learning_rate=2e-2, early_stopping_rounds=5, random_state=0)
model = XGBClassifier(n_estimators=1000, learning_rate=2e-2, early_stopping_rounds=5, random_state=0)

# Fit the model
model.fit(X_train, y_train, 
             eval_set=[(X_test, y_test)], verbose=1)

[0]	validation_0-mlogloss:1.08876


[1]	validation_0-mlogloss:1.07936
[2]	validation_0-mlogloss:1.06997
[3]	validation_0-mlogloss:1.06112
[4]	validation_0-mlogloss:1.05245
[5]	validation_0-mlogloss:1.04404
[6]	validation_0-mlogloss:1.03619
[7]	validation_0-mlogloss:1.02841
[8]	validation_0-mlogloss:1.02076
[9]	validation_0-mlogloss:1.01349
[10]	validation_0-mlogloss:1.00663
[11]	validation_0-mlogloss:1.00021
[12]	validation_0-mlogloss:0.99371
[13]	validation_0-mlogloss:0.98756
[14]	validation_0-mlogloss:0.98160
[15]	validation_0-mlogloss:0.97565
[16]	validation_0-mlogloss:0.97013
[17]	validation_0-mlogloss:0.96470
[18]	validation_0-mlogloss:0.95926
[19]	validation_0-mlogloss:0.95374
[20]	validation_0-mlogloss:0.94871
[21]	validation_0-mlogloss:0.94382
[22]	validation_0-mlogloss:0.93907
[23]	validation_0-mlogloss:0.93444
[24]	validation_0-mlogloss:0.92998
[25]	validation_0-mlogloss:0.92550
[26]	validation_0-mlogloss:0.92127
[27]	validation_0-mlogloss:0.91673
[28]	validation_0-mlogloss:0.91240
[29]	validation_0-mlogloss:0.

/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, C

[65]	validation_0-mlogloss:0.80688
[66]	validation_0-mlogloss:0.80537
[67]	validation_0-mlogloss:0.80376
[68]	validation_0-mlogloss:0.80180
[69]	validation_0-mlogloss:0.80015
[70]	validation_0-mlogloss:0.79850
[71]	validation_0-mlogloss:0.79718
[72]	validation_0-mlogloss:0.79589
[73]	validation_0-mlogloss:0.79459
[74]	validation_0-mlogloss:0.79332
[75]	validation_0-mlogloss:0.79205
[76]	validation_0-mlogloss:0.79116
[77]	validation_0-mlogloss:0.78987
[78]	validation_0-mlogloss:0.78857
[79]	validation_0-mlogloss:0.78705
[80]	validation_0-mlogloss:0.78553
[81]	validation_0-mlogloss:0.78392
[82]	validation_0-mlogloss:0.78274
[83]	validation_0-mlogloss:0.78144
[84]	validation_0-mlogloss:0.78036
[85]	validation_0-mlogloss:0.77931
[86]	validation_0-mlogloss:0.77807
[87]	validation_0-mlogloss:0.77699
[88]	validation_0-mlogloss:0.77611
[89]	validation_0-mlogloss:0.77519
[90]	validation_0-mlogloss:0.77429
[91]	validation_0-mlogloss:0.77326
[92]	validation_0-mlogloss:0.77248
[93]	validation_0-ml

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=5,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [78]:
predictions = model.predict(scaled_test)
ids = test_df['id']
reverse_mapping = {v: k for k, v in category_mapping['outcome'].items()}
submission_df = pd.DataFrame({'id': ids, 'outcome': [reverse_mapping[prediction] for prediction in predictions]})
display(submission_df.head(10))
submission_df.to_csv('data/submission_xgb_classifier.csv', index=False)

/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, C

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived
5,1240,died
6,1241,died
7,1242,died
8,1243,lived
9,1244,lived


In [158]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBClassifier

In [143]:
X = pd.read_csv('data/train.csv')
y = X['outcome']
X.drop(columns=['outcome'], inplace=True)
X_test = pd.read_csv('data/test.csv')

In [183]:
X.dtypes

id                         int64
surgery                   object
age                       object
hospital_number            int64
rectal_temp              float64
pulse                    float64
respiratory_rate         float64
temp_of_extremities       object
peripheral_pulse          object
mucous_membrane           object
capillary_refill_time     object
pain                      object
peristalsis               object
abdominal_distention      object
nasogastric_tube          object
nasogastric_reflux        object
nasogastric_reflux_ph    float64
rectal_exam_feces         object
abdomen                   object
packed_cell_volume       float64
total_protein            float64
abdomo_appearance         object
abdomo_protein           float64
surgical_lesion           object
lesion_1                   int64
lesion_2                   int64
lesion_3                   int64
cp_data                   object
dtype: object

### Category values check
Before encoding the categories with numbers, we check if some values are in the test data and not in the test data

In [145]:
categories_cols = X.select_dtypes(include='object').columns.tolist()

train_col_to_values = X[categories_cols].apply(lambda x: x.unique())
test_col_to_values = X_test[categories_cols].apply(lambda x: x.unique())

problematic_cols = []
for col in categories_cols:
    if not set(test_col_to_values[col]).issubset(set(train_col_to_values[col])):
        problematic_cols.append(col)
        print(set(train_col_to_values[col]))
        print("different values:", set(test_col_to_values[col]) - set(train_col_to_values[col]))
print("problematic_cols:", problematic_cols)


{'slight', 'mild_pain', nan, 'depressed', 'alert', 'extreme_pain', 'severe_pain'}
different values: {'moderate'}
problematic_cols: ['pain']


In [146]:
# print(X_test[X_test['pain'] == 'moderate'].id.count())
# we see only one value that is problematic, so we replace it with the closest value
X_test['pain'] = X_test['pain'].replace('moderate', 'slight')
assert X_test[X_test['pain'] == 'moderate'].id.count() == 0


### Dropping some columns
In this part we do some analysis over the columns influence on the outcome to see if we will drop some of them

In [157]:
# TODO

### Missing value handling

We deplace the missing value depending on the column.

In [185]:
# compute for each column the number of nan values
#print(X.isna().sum())
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()
# cat_cols.remove('outcome')
nan_num = X[num_cols].isna().sum()
nan_col = X[cat_cols].isna().sum()
print("missing numerical values:", nan_num[nan_num > 0])
print("missing categorical values:", nan_col[nan_col > 0])
# as we don't have too much missing values (less 20% of column value, we can replace them with the most frequent value)

cat_inputer = SimpleImputer(strategy='most_frequent')
imputed_X = pd.DataFrame(cat_inputer.fit_transform(X))
imputed_X_test = pd.DataFrame(cat_inputer.transform(X_test))

imputed_X.columns = X.columns
imputed_X_test.columns = X_test.columns

# assert we don't have any nan values
assert imputed_X.isna().sum().sum() == 0

# cinvert dtypes to original ones
imputed_X[cat_cols] = imputed_X[cat_cols].astype('category')
imputed_X_test[cat_cols] = imputed_X_test[cat_cols].astype('category')
imputed_X[num_cols] = imputed_X[num_cols].astype('float64')
imputed_X_test[num_cols] = imputed_X_test[num_cols].astype('float64')

missing numerical values: Series([], dtype: int64)
missing categorical values: temp_of_extremities       39
peripheral_pulse          60
mucous_membrane           21
capillary_refill_time      6
pain                      44
peristalsis               20
abdominal_distention      23
nasogastric_tube          80
nasogastric_reflux        21
rectal_exam_feces        190
abdomen                  213
abdomo_appearance         48
dtype: int64


### Encoding categories

In [186]:
category_enc = OrdinalEncoder()

encoded_X = imputed_X.copy(deep=False)
encoded_X_test = imputed_X_test.copy(deep=False)

# Apply ordinal encoder 
encoded_X[cat_cols] = category_enc.fit_transform(encoded_X[cat_cols])
encoded_X_test[cat_cols] = category_enc.transform(encoded_X_test[cat_cols])

# encode the target variable
outcome_enc = OrdinalEncoder()
encoded_y = outcome_enc.fit_transform(y.values.reshape(-1, 1)).flatten()
encoded_y = encoded_y.astype(int)
print("encoded_y:", encoded_y[:5])

encoded_y: [0 1 2 2 2]


### Building the Ml model

We will use an XGBClassifier in order to predict the weatlness of the horses

In [213]:
# model = XGBClassifier(n_estimators=1000, learning_rate=2e-2, early_stopping_rounds=5, random_state=0)
# model = XGBClassifier(n_estimators=1000, learning_rate=2e-2, early_stopping_rounds=100, random_state=0)
params = {'n_estimators': 400,
            'learning_rate': 0.05,
            'max_depth': 4,
            'subsample': 0.8,
            'colsample_bytree': 0.1,
            'n_jobs': -1,
            'eval_metric': 'mlogloss',
            'objective': 'multi:softprob',
            'tree_method': 'hist',
            'random_state': 42,
        }
#model = XGBClassifier(n_estimators=1000, learning_rate=1e-2, early_stopping_rounds=5, random_state=0)
model = XGBClassifier(**params)
X_train = encoded_X.drop(columns=['id']) 
X_train, X_valid = train_test_split(X_train, test_size=0.2, random_state=42)
y_train, y_valid = encoded_y[X_train.index], encoded_y[X_valid.index]
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
model.score(X_valid, y_valid)

/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, C

0.7449392712550608

In [214]:
predictions = model.predict(encoded_X_test.drop(columns=['id']))
submission_df = pd.DataFrame({'id': ids, 'outcome': [reverse_mapping[prediction] for prediction in predictions]})
display(submission_df.head(10))
submission_df.to_csv('data/submission_xgb_classifier_3.csv', index=False)

/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/douglas/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, C

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived
5,1240,died
6,1241,died
7,1242,died
8,1243,lived
9,1244,lived
